In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from imutils import paths
import numpy as np
import pickle
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from skimage import filters
from skimage.transform import resize
from skimage.feature import hog
import os
import pandas as pd
from PIL import Image


In [2]:
path = "C:/Users/ligra/Desktop/диплом/fashion-dataset/images/"
weights = [1, 0.2, 0.2, 0.4]

In [3]:
#prerparing dataset
def preparing_dataset():
    df = pd.read_csv('C:/Users/ligra/Desktop/диплом/fashion-dataset/styles.csv', error_bad_lines=False)
    #looking for classes where the number of images is < 8, so that each class falls into each of the samples
    del_index = df.articleType.value_counts().reset_index().rename(columns={'index':'articleType', 'articleType':'count'})
    del_index =del_index[del_index['count'] < 9]

    flags_buf  = df.articleType.reset_index()
    del_indexes = []

    for i in del_index.articleType:
        array_buf = flags_buf[flags_buf['articleType'] == i]
        del_indexes = array_buf['index']
        df = df.drop(del_indexes)
    
    #removing missing images from dataset
    missing_img = []
    for idx, line in df.iterrows():
        if not os.path.exists(os.path.join('C:/Users/ligra/Desktop/диплом/fashion-dataset/', 'images', str(line.id)+'.jpg')):
            missing_img.append(idx)
        
    df.drop(df.index[missing_img], inplace=True)
    del_indexes = [6695, 16194, 32309, 36381]
    df = df.drop(del_indexes)
    return df

def split_clasters(df):
    claster_names = {}
    for claster_name in df.articleType:
        df_id = df[df['articleType'] == claster_name]
        list_id = list(df_id.reset_index().id)
        claster_names[claster_name] = list_id
    return claster_names  

In [4]:
#image processing functions
def hsv(num):
    img = np.load('C:/Users/ligra/Documents/GitHub/recommendation_system/img_resize/'+str(num)+'.npy')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    return hist

def hog_f(num):
    img = np.load('C:/Users/ligra/Documents/GitHub/recommendation_system/img_resize/'+str(num)+'.npy')
    hog_img, fd = hog(img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return hog_img.reshape(hog_img.shape[0],1)

def sobel(num):
    img = np.load('C:/Users/ligra/Documents/GitHub/recommendation_system/img_resize/'+str(num)+'.npy')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobel_img = filters.sobel(img)
    sobel_img  = filters.gaussian(sobel_img, sigma=2.0)
    return sobel_img.flatten()

def texture(num):
    img = np.load('C:/Users/ligra/Documents/GitHub/recommendation_system/img_resize/'+str(num)+'.npy')
    g_kernel = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    texture_img = cv2.filter2D(img, cv2.CV_8UC3, g_kernel)
    return texture_img.flatten()



In [5]:
#getting features
def get_features(img_num):
    hsv_vector = hsv(img_num)
    sobel_vector = sobel(img_num)
    texture_vector = texture(img_num)
    hog_vector = hog_f(img_num)  
    return [hsv_vector, sobel_vector, texture_vector, hog_vector]


def features_dataset(df):
    features = {}
    for claster_name in df:
        if claster_name not in features:
            features[claster_name] = {}
        for img_num in df[claster_name]:
            features[claster_name][img_num] = get_features(img_num)
    return features

In [6]:
#functions for saving and loading features datasets
def save_in_pickle(data, pickle_file):
    file = open(pickle_file,"wb")
    pickle.dump(data,file)
    file.close()
    
def load_from_pickle(pickle_file):
    file = open(pickle_file,'rb')
    pickle_data = pickle.load(file)
    file.close()
    return pickle_data

In [7]:
#image similarity assessment
def cosine(a,b):
    dot = np.dot(a.T, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = (dot / norma) / normb
    return cos

def cosine_score(test_features, train_features):
    score = 0
    for num_feature in range(len(test_features)):
        if len(test_features[num_feature]) != len(train_features[num_feature]):
            min_size = min(len(test_features[num_feature]), len(train_features[num_feature]))
            test_features[num_feature] = test_features[num_feature][0:min_size]
            train_features[num_feature] = train_features[num_feature][0:min_size]
        score += cosine(train_features[num_feature],test_features[num_feature])  
    return score/4
   

In [8]:
#main function
def recommend_images(img_test):
    lead_similarity = {}
    test_features = get_features(img_test)
    train_classes = list(train_features.keys())
    for key in train_classes:
        id_img = list(train_features[key].keys())
        if id_img != []:
            buf_features = train_features[key][id_img[0]]
            buf_score = cosine_score(test_features, buf_features)
            lead_similarity[key] = buf_score
    
    #selection of top 3 similar images among class leaders
    lead_score = []
    for key in lead_similarity:
        lead_score.append(lead_similarity[key])
    
    lead_score.sort(reverse=True)
    top_lead = []
    for key in lead_similarity:
        if lead_similarity[key] == lead_score[0] or lead_similarity[key] == lead_score[1] or lead_similarity[key] == lead_score[2]:
            top_lead.append(key)
    
   #selection top 3 images among all 
    top3_score = {}
    
    for type_name in top_lead:
        id_img = list(train_features[type_name].keys())
        buf_score_array = []
        if id_img != []:
            for i in  range(len(id_img)):
                buf_features = train_features[type_name][id_img[i]]
                buf_score = cosine_score(test_features, buf_features)
                buf_score_array.append(buf_score)
        
        avg_score = sum(buf_score_array) / len(buf_score_array)
        top3_score[type_name] = avg_score
    
    max_score = 0
    result = ''
    
    for key in top3_score:
        if top3_score[key] > max_score:
            max_score = top3_score[key]
            result = key
    
    return result

In [9]:
def scatter_plot(accuracy_processed_img, accuracy_all_img):
    x = [i for i in range(len(accuracy_processed_img))]
    plt.plot(x, accuracy_processed_img, label='precision', color = 'red')
    plt.plot(x, accuracy_all_img, label='recall', color = "green")
    plt.xlabel('X input')
    plt.ylabel("Y output")
    plt.legend()
    plt.show()

In [11]:
df = preparing_dataset()

C:\Users\ligra\AppData\Local\Temp/ipykernel_14656/3924117327.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  df = preparing_dataset()
b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 3574

In [13]:
tags_train, tags_test = train_test_split(df, test_size=0.2, random_state=1, stratify=df['articleType'])

In [14]:
tags_train = split_clasters(tags_train)
tags_test = split_clasters(tags_test)

In [15]:
#creating file with features
train_features = features_dataset(tags_train)
save_in_pickle(train_features, "train_features")

In [16]:
#loading file with features
train_features = load_from_pickle("train_features")

In [17]:
#test
k = 4
accuracy_processed_img = []
accuracy_all_img = []
proccesed_imgs = 0
correct_answer = 0
all_score = 0


for claster_name in tags_test:
    for img_test in tags_test[claster_name]:
        try:
            result = recommend_images(str(img_test))

            if claster_name == result :
                correct_answer += 1
            proccesed_imgs += 1
            #the number of correct definitions in relation to the number of processed images
            accuracy_processed_img.append(correct_answer/proccesed_imgs)
            #the number of correct definitions in relation to the number of all images
            accuracy_all_img.append(correct_answer/len(tags_test))
        
        except:
            pass
            print('ERROR')


C:\Users\ligra\AppData\Local\Temp/ipykernel_14656/40272651.py:6: RuntimeWarning: invalid value encountered in double_scalars
  cos = (dot / norma) / normb
C:\Users\ligra\AppData\Local\Temp/ipykernel_14656/40272651.py:6: RuntimeWarning: invalid value encountered in true_divide
  cos = (dot / norma) / normb


In [1]:
#accuracy output
print(correct_answer/proccesed_imgs)

0.47168021680216804
